<a href="https://colab.research.google.com/github/alyssacampos3/Fitness_App/blob/main/User.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving fake_fitness_users_with_tone_restrictions.csv to fake_fitness_users_with_tone_restrictions.csv


In [ ]:
import pandas as pd
df = pd.read_csv("fake_fitness_users_with_tone_restrictions.csv")


In [20]:
import numpy as np
import random

# Sample exercise database by goal and targeted body parts
exercise_db = {
    "lose": {
        "full_body": ["Jumping jacks", "Mountain climbers", "Burpees"],
        "core": ["Plank", "Bicycle crunches"],
        "legs": ["Bodyweight squats", "Lunges"],
    },
    "gain": {
        "chest": ["Push-ups", "Incline dumbbell press"],
        "arms": ["Bicep curls", "Tricep dips"],
        "legs": ["Barbell squats", "Deadlifts"],
    },
    "maintain": {
        "full_body": ["Jump rope", "Bodyweight circuits"],
        "core": ["Sit-ups", "Russian twists"],
    },
    "tone": {
        "glutes": ["Glute bridges", "Donkey kicks"],
        "arms": ["Resistance band curls", "Lateral raises"],
        "core": ["Leg raises", "Plank taps"]
    }
}

# Body part keywords to match exercises
exercise_parts = {
    "left_shoulder": ["shoulder", "raises", "press"],
    "right_shoulder": ["shoulder", "raises", "press"],
    "upper_back": ["row", "pull", "deadlift"],
    "lower_back": ["deadlift", "good morning", "superman"],
    "left_leg": ["squat", "lunge", "leg"],
    "right_leg": ["squat", "lunge", "leg"],
    "left_ankle": ["jump", "lunge", "step"],
    "right_ankle": ["jump", "lunge", "step"],
    "left_wrist": ["curl", "push-up"],
    "right_wrist": ["curl", "push-up"],
    "left_foot": ["jump", "step"],
    "right_foot": ["jump", "step"],
    "neck": ["shrug"]
}

# Restriction checker helper
def is_exercise_safe(exercise, restrictions):
    for part in restrictions:
        for keyword in exercise_parts.get(part, []):
            if keyword.lower() in exercise.lower():
                return False
    return True

# Main function to generate workout plan
def generate_workout(user_input, model, label_encoders):
    # Format input into numpy array
    input_arr = np.array([user_input])

    # Predict goal
    pred = model.predict(input_arr)
    goal = label_encoders['goal'].inverse_transform([pred.argmax()])[0]

    # Get user restrictions
    restriction_flags = user_input[4:]  # Assuming first 4 = age, gender, fitness_level, weight
    restriction_keys = list(exercise_parts.keys())
    restrictions = [restriction_keys[i] for i, val in enumerate(restriction_flags) if val == 1]

    # Pick exercises that match the goal and are safe
    goal_exercises = exercise_db[goal]
    workout_plan = []

    for category, ex_list in goal_exercises.items():
        safe_exercises = [ex for ex in ex_list if is_exercise_safe(ex, restrictions)]
        if safe_exercises:
            workout_plan.append(f"{random.choice(safe_exercises)} - 3 sets x 12 reps")

    return goal, workout_plan


In [ ]:
from faker import Faker
import random
import pandas as pd

fake = Faker()

goals = ['lose fat', 'gain muscle', 'maintain', 'Tone']
levels = ['beginner', 'intermediate', 'advanced']
genders = ['male', 'female', 'non-binary']
body_parts = [
    'left_shoulder', 'right_shoulder', 'upper_back', 'lower_back',
    'left_leg', 'right_leg', 'left_ankle', 'right_ankle',
    'left_wrist', 'right_wrist', 'left_foot', 'right_foot', 'neck'
]

def generate_restrictions():
    # 30% of users have 1–3 restrictions
    if random.random() < 0.3:
        return random.sample(body_parts, k=random.randint(1, 3))
    return []

data = []

for _ in range(1000):
    restrictions = generate_restrictions()
    user = {
        'name': fake.name(),
        'age': random.randint(18, 60),
        'gender': random.choice(genders),
        'goal': random.choice(goals),
        'fitness_level': random.choice(levels),
        'days_per_week': random.randint(2, 6),
        'restrictions': ','.join(restrictions)
    }
    data.append(user)

df = pd.DataFrame(data)
df.to_csv("fake_fitness_users_with_restrictions.csv", index=False)


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# Load your dataset
df = pd.read_csv("fake_fitness_users_with_tone_restrictions.csv")

# Encode categorical columns
label_encoders = {}
for col in ['gender', 'fitness_level', 'goal']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# One-hot encode restrictions
body_parts = [
    'left_shoulder', 'right_shoulder', 'upper_back', 'lower_back',
    'left_leg', 'right_leg', 'left_ankle', 'right_ankle',
    'left_wrist', 'right_wrist', 'left_foot', 'right_foot', 'neck'
]

for part in body_parts:
    df[part] = df['restrictions'].apply(lambda x: 1 if part in x else 0)

# Drop non-numerical columns
df = df.drop(['name', 'restrictions'], axis=1)

# Split data
X = df.drop('goal', axis=1)
y = df['goal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build the model
model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# Compile and train
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

# Save the model
model.save("fitness_restriction_model_with_tone.keras")



Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2650 - loss: 1.7347 - val_accuracy: 0.3000 - val_loss: 1.4234
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2682 - loss: 1.4331 - val_accuracy: 0.2500 - val_loss: 1.4136
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2731 - loss: 1.3899 - val_accuracy: 0.2625 - val_loss: 1.4232
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2836 - loss: 1.3981 - val_accuracy: 0.2750 - val_loss: 1.3908
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2799 - loss: 1.3933 - val_accuracy: 0.2500 - val_loss: 1.3778
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2373 - loss: 1.4071 - val_accuracy: 0.2250 - val_loss: 1.4090
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2608 - loss: 1.3894 - val_accuracy: 0.2500 - val_loss: 1.3835
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2753 - loss: 1.3862 - val_accuracy: 0.1875 - val_loss: 1.4540
E

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 goals: lose, gain, maintain
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

# Save
model.save("fitness_restriction_model.keras")


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2615 - loss: 2.2599 - val_accuracy: 0.2500 - val_loss: 1.5373
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2532 - loss: 1.4645 - val_accuracy: 0.2375 - val_loss: 1.4158
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2291 - loss: 1.4119 - val_accuracy: 0.2500 - val_loss: 1.4048
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2745 - loss: 1.3976 - val_accuracy: 0.1875 - val_loss: 1.4761
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2904 - loss: 1.3880 - val_accuracy: 0.2125 - val_loss: 1.3990
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2483 - loss: 1.3993 - val_accuracy: 0.2250 - val_loss: 1.4173
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2765 - loss: 1.3864 - val_accuracy: 0.2875 - val_loss: 1.4093
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2682 - loss: 1.4024 - val_accuracy: 0.2000 - val_loss: 1.4232
Ep

In [ ]:
import numpy as np

# Example input (just like yours, but formatted)
X_input = np.array([[29, 0, 2, 4, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

# Make prediction
prediction = model.predict(X_input)
goal = label_encoders['goal'].inverse_transform([prediction.argmax()])[0]

print("Predicted goal:", goal)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Predicted goal: gain muscle


In [ ]:
# User inputs
user_input = {
    'age': 29,
    'gender': 'female',
    'fitness_level': 'intermediate',
    'days_per_week': 4,
    'restrictions': ['left_leg', 'lower_back']
}

# Encode gender and level
gender = label_encoders['gender'].transform([user_input['gender']])[0]
level = label_encoders['fitness_level'].transform([user_input['fitness_level']])[0]

# One-hot restrictions
restriction_vector = [1 if part in user_input['restrictions'] else 0 for part in body_parts]

# Final input
X_input = [[
    user_input['age'],
    gender,
    level,
    user_input['days_per_week'],
    *restriction_vector
]]

# Predict
model = tf.keras.models.load_model("fitness_restriction_model.keras")
prediction = model.predict(X_input)
goal = label_encoders['goal'].inverse_transform([prediction.argmax()])[0]


ValueError: Unrecognized data type: x=[[29, np.int64(0), np.int64(2), 4, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]] (of type <class 'list'>)

In [ ]:
!pip install tensorflow


In [ ]:
!pip install faker pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00


In [ ]:
!pip install matplotlib-venn